O nosso objetivo é de prever o número de *sets* que um jogo de ténis profissional de Austrália vai ter, baseado num número de recursos que um jogo vai ter. 

Um jogo de ténis é composto por pontos, jogos e sets. De acordo com as [regras de ténis da Federação Internacional de Tennis (IFT)](https://www.atptour.com/en/corporate/rulebook), um set é uma pontuação, onde o primeiro jogador a ganhar 6 jogos ganha um set, sendo um jogo decidido pelo número de sets que um jogador ganha. Um jogo pode ser à melhor de 3 ou à melhor de 5, ou seja, o jogador precisa de ganhar 2 sets (o que traduz para 3 sets de máximo) ou 3 sets (máximo 5), respectivamente.  O jogador que decide o lado que começa/quem serve primeiro é decidido aleatoriamente, sendo que o outro decide a outra decisão.

Baseado nestes factos, parece que o projeto vai ser de regressão ordinal, sendo que o número de sets varia entre 2 e 5, não mais e não menos.

In [ ]:
if (FALSE){
    filePath <-here("data", "atpplayers.json")
    dest <- here("data", "atpRaw.parquet")

    print(paste("Getting", filePath))
    ndjson::stream_in(filePath) -> atp

    print(paste("Writing parquet to", dest))
    atp %>% mutate(
        player.bornAt = Born,
        tournament.date = Date,
        opponent.rank = GameRank,
        match.tournamentRound = GameRound,
        tournament.ground = Ground,
        player.hand = Hand,
        player.height = Height,
        player.link = LinkPlayer,
        tournament.location = Location,
        opponent.name = Oponent,
        player.name = PlayerName,
        tournament.prize = Prize,
        match.sets = Score,
        tournament.name = Tournament,
        match.WL = WL,
        .keep = "none"
    ) %>% select(order(colnames(.))) -> atpp
    atpp %>% arrow::write_parquet(dest)
}
arrow::read_parquet(here("data", "atpRaw.parquet"), as_data_frame = TRUE) -> atp
head(atp)

match.sets,match.tournamentRound,match.WL,opponent.name,opponent.rank,player.bornAt,player.hand,player.height,player.link,player.name,tournament.date,tournament.ground,tournament.location,tournament.name,tournament.prize
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
64 62,Semi-Finals,W,Marin Cilic,30,"Belgrade, Serbia","Right-Handed, Two-Handed Backhand",188,https://www.atptour.com/en/players/novak-djokovic/d643/player-activity?year=all&matchType=Singles,Novak Djokovic,2021.11.22 - 2021.12.05,Hard,"Madrid, Spain",Davis Cup Finals,
63 76,Round of 16,W,Karen Khachanov,26,"Belgrade, Serbia","Right-Handed, Two-Handed Backhand",188,https://www.atptour.com/en/players/novak-djokovic/d643/player-activity?year=all&matchType=Singles,Novak Djokovic,2022.02.21 - 2022.02.26,Hard,"Dubai, U.A.E.",Dubai,"$2,794,840"
62 61,Round Robin,W,Cameron Norrie,12,"Belgrade, Serbia","Right-Handed, Two-Handed Backhand",188,https://www.atptour.com/en/players/novak-djokovic/d643/player-activity?year=all&matchType=Singles,Novak Djokovic,2021.11.15 - 2021.11.21,Hard,"Turin, Italy",Nitto ATP Finals,"$7,250,000"
63 62,Round Robin,W,Andrey Rublev,5,"Belgrade, Serbia","Right-Handed, Two-Handed Backhand",188,https://www.atptour.com/en/players/novak-djokovic/d643/player-activity?year=all&matchType=Singles,Novak Djokovic,2021.11.15 - 2021.11.21,Hard,"Turin, Italy",Nitto ATP Finals,"$7,250,000"
46 63 63,Finals,W,Daniil Medvedev,2,"Belgrade, Serbia","Right-Handed, Two-Handed Backhand",188,https://www.atptour.com/en/players/novak-djokovic/d643/player-activity?year=all&matchType=Singles,Novak Djokovic,2021.11.01 - 2021.11.07,Hard,"Paris, France",ATP Masters 1000 Paris,"�5,207,405"
36 60 76,Semi-Finals,W,Hubert Hurkacz,10,"Belgrade, Serbia","Right-Handed, Two-Handed Backhand",188,https://www.atptour.com/en/players/novak-djokovic/d643/player-activity?year=all&matchType=Singles,Novak Djokovic,2021.11.01 - 2021.11.07,Hard,"Paris, France",ATP Masters 1000 Paris,"�5,207,405"


In [ ]:
nrow(atp)

[1] 1308835

A base de dados mostra informação pública de [ATP Tour](https://www.atptour.com/en/players).

A primeira vista dos dados revela vários factos:

* A base de dados tem 1308835, sendo cada registro um jogo de um jogador "principal" (com as informações de tal) contra um oponente (sem as informações de tal).
    * Isto poderá indicar que teremos de transformar os dados em registros de jogos entre os dois jogadores, sem jogador principal. Não temos a certeza se isto é necessário.
* Ao fazer a cabeça dos dados, parece que apenas mostra o jogador Novak Djokovic como principal. Isto pode indicar que a base de dados pode estar ordenada de uma certa forma.
    * Esta ordem apenas pode indicar como os dados foram obtidos, e nada sobre um registro específico. Informação sobre a recolha pode ajudar a perceber melhor o contexto da base de dados.
* A data parece mostrar as datas do torneio e não do jogo especificamente
* A coluna `prize` parece mal formatada, com valores omissos presentes. Da mesma forma, aprece que os valores estão em várias moedas diferentes.
* A coluna `GameRank` parece indicar o *rank* do oponente, não o rank do jogo.
    * Desta forma, desde que o oponente apareça também como principal, consegue-se ter o rank de ambas as pessoas. No entanto, isto não é garantido.
    * Nos casos que isto acontece, pode ajudar fazer a diferença de rank entre jogadores.
* A coluna `Hand` parece incluir ambas as informações do *backhand* e se é destro ou esquerdino
* O score parece conter a pontuação de cada set, sendo ai onde vamos buscar o número de sets.

In [ ]:
set.seed(1)
atp %>% slice_sample(n=5)

Born,Date,GameRank,GameRound,Ground,Hand,Height,LinkPlayer,Location,Oponent,PlayerName,Prize,Score,Tournament,WL,_id.$oid
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
,2010.08.02 - 2010.08.08,-,Semi-Finals,Clay,null,NA,https://www.atptour.com/en/players/juan-lizariturry/l996/player-activity?year=all&matchType=Singles,Spain,Taro Daniel,Juan Lizariturry,"$10,000",63 62,Spain F28,W,624ab365a5c952847318a12b
"Sendai, Japan",2017.03.13 - 2017.03.19,164,Round of 16,Hard,"Right-Handed, Two-Handed Backhand",175,https://www.atptour.com/en/players/yuichi-sugita/se73/player-activity?year=all&matchType=Singles,"Shenzhen, China",Jan Satral,Yuichi Sugita,"$75,000","67, 63 62",Shenzhen,W,624ab34f936e4d97ffee0700
,2005.11.21 - 2005.11.27,1367,Round of 32,Hard,null,NA,https://www.atptour.com/en/players/ludwig-pellerin/pa58/player-activity?year=all&matchType=Singles,"Monastin, Tunisia",Anas Fattar,Ludwig Pellerin,"$10,000",64 64,Tunisia F5,W,624ab40fba7043df73e7f9e9
"Voorhees, NJ, USA",2015.04.20 - 2015.04.26,462,2nd Round Qualifying,Clay,"Right-Handed, Two-Handed Backhand",185,https://www.atptour.com/en/players/tommy-paul/pl56/player-activity?year=all&matchType=Singles,"Savannah, GA, U.S.A.",Andrea Collarini,Tommy Paul,"$50,000","76, 62",Savannah,W,624ab34a13b144c54b3cfd0e
"Santiago, Chile",2007.03.19 - 2007.04.01,107,1st Round Qualifying,Hard,"Right-Handed, Two-Handed Backhand",191,https://www.atptour.com/en/players/paul-capdeville/c634/player-activity?year=all&matchType=Singles,"Miami, FL, U.S.A.",Andrei Pavel,Paul Capdeville,"$3,200,000",26 63 75,ATP Masters 1000 Miami,W,624ab37573539d8f27174222


Estes dados mostram também que há jogadores principais sem informações.

# Colunas

## PlayerName

In [ ]:
atp %>% distinct(PlayerName)

PlayerName
<chr>
Novak Djokovic
Daniil Medvedev
Rafael Nadal
Alexander Zverev
Stefanos Tsitsipas
Matteo Berrettini
Andrey Rublev
Casper Ruud
Felix Auger-Aliassime


# Paises
TODO analise paises

## Date

TODO dates

# Características
TODO